## Scratchpad to use for Dive into Deep Learning (D2L) book
Using this as scratchpad to teach

In [65]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
# Use apple silicon mps
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
device = mps_device
print(device)

mps


## Torch autograd - L2

In [7]:
a = torch.tensor([2., 3], requires_grad=True)
b = torch.tensor([6., 4], requires_grad=True)

In [8]:
Q = 3*a**3 - b**2

In [9]:
external_grad = torch.tensor([1., 1])
Q.backward(gradient=external_grad)

In [10]:
print(a.grad, b.grad)

tensor([36., 81.]) tensor([-12.,  -8.])


In [19]:
a.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [23]:
Q = 3*a**3 - b**2
external_grad = torch.tensor([1., 0])
Q.backward(gradient=external_grad) # J.T * v

In [24]:
print(a.grad, b.grad)

tensor([72.,  0.]) tensor([-24.,   0.])


## Building blocks - L3

In [30]:
from torch.utils import data

def synthesize_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = X @ w + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2., -3.4])
true_b = 4.2
features, labels = synthesize_data(true_w, true_b, 1000)
features.shape, labels.shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [33]:
def load_data(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [34]:
batch_size = 10
data_iter = load_data((features, labels), batch_size)

In [42]:
from torch import nn

net = nn.Sequential(
    nn.Linear(2, 1)
)
net[0].weight, net[0].bias

(Parameter containing:
 tensor([[-0.2316, -0.6634]], requires_grad=True),
 Parameter containing:
 tensor([0.6364], requires_grad=True))

In [50]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
net[0].weight, net[0].bias

(Parameter containing:
 tensor([[ 0.0159, -0.0125]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True))

In [55]:
loss = nn.MSELoss()
optim = torch.optim.SGD(net.parameters(), lr=0.02)

In [56]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        optim.zero_grad()
        
        l = loss(net(X), y)
        l.backward()
        optim.step()
    l = loss(net(features), labels)
    print(f'epoch -> {epoch + 1} with loss -> {l:f}')
        

epoch -> 1 with loss -> 0.000100
epoch -> 2 with loss -> 0.000100
epoch -> 3 with loss -> 0.000100


In [63]:
w = net[0].weight.data
print(abs(true_w - w.reshape(true_w.shape)))

tensor([0.0013, 0.0002])


In [64]:
b = net[0].bias.data
print(abs(true_b - b))

tensor([0.0004])
